# Tokenize Openwebtext

In [ ]:
# Download Dataset
import datasets
from datasets import load_dataset
dataset = load_dataset("openwebtext")

In [ ]:
# Load Tokenizer and Get Vocab Size

#show how many documents in this dataset
from transformers import GPT2TokenizerFast

# You can choose another tokenizer if you want
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
print("Gpt2 Tokenizer Vocab Size:"+ str(tokenizer.vocab_size))
print(tokenizer.is_fast)

def tokenize_map_function(items):
    # iterate each item in a batch
    # print(type(items["text"]))
    global tokenizer
    return tokenizer(items["text"],max_length=10000)

tokenized_dataset = dataset.map(tokenize_map_function, batched = True)

print(tokenized_dataset.column_names)

In [4]:
# save this dataset for future usage
tokenized_dataset.save_to_disk("./tokenized_dataset")

In [ ]:
# load back
tokenized_dataset.load_from_disk("./tokenized_dataset")

In [10]:
tokenized_dataset = tokenized_dataset['train']

## Write tokenized Dataset to a binary file

In [14]:
import numpy as np
encodedFile = "./openwebtext_gpt2.bin"
encodedSeqF = open(encodedFile,'wb')

def write2BinaryFile(items):
    global encodedSeqF
#     written_bytes = b''
    for tokens in items['input_ids']:
        tokens_len =int(len(tokens))
        # write its length
#         written_bytes += tokens_len.to_bytes(4,byteorder='little',signed=True)
#         written_bytes +=np.array(tokens,dtype=np.uint32).tobytes() 
        encodedSeqF.write(tokens_len.to_bytes(4,byteorder='little',signed=True))
        # write the list of tokens
        encodedSeqF.write(np.array(tokens,dtype=np.uint32).tobytes())
#     encodedSeqF.write(written_bytes) 

# use map function of dataset to write each tokens into binary file
tokenized_dataset = tokenized_dataset.map(write2BinaryFile, batched = True)
encodedSeqF.close()

100%|███████████████████████████| 8014/8014 [2:10:39<00:00,  1.02ba/s]
